## A simple DMET script for 1D Hubbard-Like model

In [13]:
!hostname

amarel1.amarel.rutgers.edu


In [25]:
import numpy as np
from pyscf import fci 
from scipy.optimize import minimize # for chem potential minimization

In [15]:
# Setting up the system
norb = 6
nimp = 2 # impurity orbitals
nemb = nimp*2
ne = norb//2
ne_emb = nimp
ne_imp = ne // (norb // nimp)
U = 4

In [32]:
# Hamiltonian Hubbard 1D, pbc, one spin case
h1e = np.zeros((norb, norb))
h2e = np.zeros((norb,)*4)
for i in range(norb):
    h1e[i, (i+1)%norb] = -1
    h1e[i, (i-1)%norb] = -1
    h2e[i,i,(i+1)%norb,(i+1)%norb] = U # We need off-diagonal U because diagonal U only works on alpha-beta, and we only have alpha electrons
    h2e[(i+1)%norb,(i+1)%norb, i, i] = U


In [33]:
# Initialize correlation potential
u_corr0 = np.zeros((nimp, nimp))

In [34]:
# add correlation potential to the 1-body Hamiltonian
def add_u_corr(h1e, u_corr, nimp):
    n_cell = norb // nimp 
    h1e_new = np.copy(h1e)
    for x in range(n_cell):
        h1e_new[x*nimp:(x+1)*nimp, x*nimp:(x+1)*nimp] += u_corr 
    return h1e_new 

h1e_new = add_u_corr(h1e, u_corr0, nimp)

In [35]:
# Solve low-level Hamiltonian and evaluate the rdm1
ew, ev = np.linalg.eigh(h1e_new)
orb_occ = ev[:, :ne]
rdm1 = np.dot(orb_occ, orb_occ.T)

In [36]:
# construct bath from rdm1
rdm1_mix = rdm1[nimp:, :nimp]
bath, s, _ = np.linalg.svd(rdm1_mix, full_matrices=False) # s should show how correlated A and E are

In [37]:
# construct rotation (projection) matrix to the embedding space
rotmat = np.zeros((norb, nimp*2))
rotmat[:nimp, :nimp] = np.eye(nimp)
rotmat[nimp:, nimp:] = bath 

In [38]:
# construct embedding Hamiltonian with interacting bath formulation
h1e_emb = np.dot(np.dot(rotmat.T.conj(), h1e), rotmat) 
h2e_emb = np.einsum('pi, qj, pqrs, rk, sl ->ijkl', rotmat.conj(), rotmat.conj(), h2e, rotmat, rotmat) # using pyscf tradition c^+ c^+ c c


In [39]:
# fci solver for spinless case (only have alpha electrons)

def fci_spinless(h1e, eri, norb, ne):
    nelec = (ne, 0)
    cisolver = fci.direct_spin1
    e, c = cisolver.kernel(h1e, eri, norb, nelec)
    rdm1 = cisolver.make_rdm1(c, norb, nelec)
    return e, rdm1 

In [59]:
# add a global chemical potential to make sure the impurity electron  = ne_imp
# we need to optimize the chemical potential
mu_global0 = 0.0
def diff_ne_imp(mu):
    mu_glob_mat = np.zeros((nemb, nemb))
    mu_glob_mat[:nimp, :nimp] = np.eye(nimp)*mu
    h1e_emb_new = h1e_emb - mu_glob_mat
    _, rdm1_emb = fci_spinless(h1e_emb_new, h2e_emb, 2*nimp, ne_emb)
    ne_imp_n = np.sum(np.diag(rdm1_emb)[:nimp])
    ne_diff = abs(ne_imp_n - ne_imp)
    print(f"Electron number diff: {ne_diff}")
    return ne_diff

# optimize mu_global to minimize diff_ne_imp
mu_global = minimize(diff_ne_imp, x0=mu_global0, method="Powell", tol=1e-3, options={'maxiter': 2}).x
print(f"Optimized global chemical potential (mu_global): {mu_global}")

Electron number diff: 0.10359515664914176
Electron number diff: 0.10359515664914176


Electron number diff: 0.020309221567981206
Electron number diff: 0.10942570812723629
Electron number diff: 0.02458160943051424
Electron number diff: 0.0492622757392186
Electron number diff: 0.0009211635109995786
Electron number diff: 0.008226598813688923
Electron number diff: 0.010075294278874436
Electron number diff: 0.10107680500695748
Electron number diff: 0.0009211635109995786
Electron number diff: 0.10107680500695748
Electron number diff: 0.19808641328979015
Electron number diff: 0.0600597438626882
Electron number diff: 0.03453528471493161
Electron number diff: 0.005361470342082786
Electron number diff: 0.006890484878522285
Electron number diff: 0.0003301875858467973
Electron number diff: 0.0022482337453553214
Electron number diff: 0.00020505322484343225
Electron number diff: 0.00011485170824543367
Electron number diff: 0.00042283231639173025
Electron number diff: 3.422158194488212e-05
Electron number diff: 5.447153740356647e-05
Electron number diff: 3.422158194488212e-05
Electron

In [61]:
#solve the embedding Hamiltonian
mu_glob_mat = np.zeros((nemb, nemb))
mu_glob_mat[:nimp, :nimp] = np.eye(nimp)*mu_global
h1e_emb_new = h1e_emb - mu_glob_mat
e, rdm1_emb = fci_spinless(h1e_emb_new, h2e_emb, 2*nimp, ne_emb)

In [66]:
# Next compare the difference between the two RDM1s
rdm1_latt_emb = np.dot(np.dot(rotmat.T, rdm1), rotmat)
# We only compare the impurity part of rdm1
rdm1_tol = 1e-5
rdm1_imp_diff = rdm1_emb[:nimp, :nimp] - rdm1_latt_emb[:nimp, :nimp] 
abs_rdm1_imp_diff = np.linalg.norm(rdm1_imp_diff)
if abs_rdm1_imp_diff < rdm1_tol:
    print("RDM1 are the same, DMET converged")
else:
    print(f"RDM1 diff = {abs_rdm1_imp_diff}, we need to change the correlation potential!")

RDM1 diff = 0.0937182295822739, we need to change the correlation potential!


### Deriving the gradient for optimizing the correlation potential
We want to minimize
$$f(u) = (D_{\text{FCI}} - D_{\text{HF}}(u))^2$$
The gradient is
$$\frac{\partial f(u)}{\partial u_ij} = $$